In [1]:
import tweetscrape
import analysicle
import pandas as pd
import requests

In [2]:
twitter_id_key = {
    51241574 : 'ap_news',
    7587032 : 'skynews_UK',
    16834659 : 'skynews_AU',
    87818409 : 'guardian',
    16675569 : 'smh',
    2878895652 : 'betoota',
    15250661 : 'newscom',
    1652541 : 'reuters',
    2768501 : 'abc',
    8197942 : 'sbs'
}

In [3]:
def analyse(twitter_id):
    # scrape tweets
    new_tweets = tweetscrape.main(twitter_id)
    print('num of tweets: ' + str(len(new_tweets)))
    
    # analyse tweets
    publication = twitter_id_key[twitter_id]
    
    # success report for analysis
    report = {
            'Success' : 0,
            'Null' : 0,
            'Deleted' : 0
        }
    
    for tweet, details in list(new_tweets.items()):
    
        # pull url from tweet scrape and run analysis
        my_url = details.get('url')
        analysis = eval(f'''analysicle.{publication}('{my_url}')''')
        
        # check if not_news_alert == False
        if analysis[2] == False:
            new_scores = analysis[1]
            details.update(new_scores)
            # check if blank_alert == False
            if analysis[0] == False:
                report['Success'] += 1
            else:
                report['Null'] += 1
        else:
            # delete entries where not_news_alert == True
            new_tweets.pop(tweet)
            report['Deleted'] += 1
    
    print(f'''Success: {report['Success']}''')
    print(f'''Nulls Present: {report['Null']}''')
    print(f'''Deleted Tweet: {report['Deleted']}''')
    
    return new_tweets

In [84]:
def update_df(tweets):
    
    # upload data bank
    data = pd.read_csv('news_bank.csv')
    
    # transpose tweet dict into df
    new_data = pd.DataFrame(tweets).T
    
    # move tweet_id out of index column and re-type as string
    new_data = new_data.reset_index().rename(columns={'index':'tweet_id'})
    new_data['tweet_id'] = new_data['tweet_id'].astype('|S')
    
    # concatenate dataframes, clean index
    df = pd.concat([data, new_data])
    df.reset_index(drop=True, inplace=True)
    
    # export to csv
    df.to_csv('news_bank.csv')
    
    # print success report
    print(f'{str(new_data.shape[0])} new rows added to news_bank')

In [95]:
for twitter_id in twitter_id_key:
    data = analyse(twitter_id)
    update_df(data)
    print(twitter_id_key[twitter_id] + ' - Success!')

200
num of tweets: 7
Success: 5
Nulls Present: 2
Deleted Tweet: 0
7 new rows added to news_bank
ap_news - Success!
200
num of tweets: 10
Success: 6
Nulls Present: 4
Deleted Tweet: 0
10 new rows added to news_bank
skynews_UK - Success!
200
num of tweets: 10
Success: 4
Nulls Present: 6
Deleted Tweet: 0
10 new rows added to news_bank
skynews_AU - Success!
200
num of tweets: 10
Success: 4
Nulls Present: 5
Deleted Tweet: 1
9 new rows added to news_bank
guardian - Success!
200
num of tweets: 10
Success: 8
Nulls Present: 2
Deleted Tweet: 0
10 new rows added to news_bank
smh - Success!
200
num of tweets: 9
Success: 8
Nulls Present: 0
Deleted Tweet: 1
8 new rows added to news_bank
betoota - Success!
200
num of tweets: 8
Success: 8
Nulls Present: 0
Deleted Tweet: 0
8 new rows added to news_bank
newscom - Success!
200
num of tweets: 8
Success: 8
Nulls Present: 0
Deleted Tweet: 0
8 new rows added to news_bank
reuters - Success!
200
num of tweets: 10
Success: 3
Nulls Present: 7
Deleted Tweet: 0
10 